# EDA

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#import geopandas as gpd
#import plotly.express as px
#import calendar
#import descartes
import os, sys

from matplotlib.ticker import FuncFormatter
#from shapely.geometry import Point, Polygon

In [2]:
os.chdir("../..")
os.chdir("src/utils")

In [3]:
import eda

In [4]:
data = pd.read_csv('../../data/Rodent_Inspection.csv')

In [5]:
data.head()

,INSPECTION_TYPE,JOB_TICKET_OR_WORK_ORDER_ID,JOB_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,HOUSE_NUMBER,STREET_NAME,ZIP_CODE,X_COORD,Y_COORD,LATITUDE,LONGITUDE,BOROUGH,INSPECTION_DATE,RESULT,APPROVED_DATE,LOCATION
0,Initial,13252142,PC7732347,1,2024090050,2,2409,50,0000,MELROSE AVENUE,NaN,NaN,NaN,0.0,0.0,Bronx,07/07/2021 10:05:45 AM,Failed for Other R,07/09/2021 11:33:07 AM,"(0.0, 0.0)"
1,Compliance,12802045,PC7500960,2,5011840160,5,1184,160,000,WENLOCK STREET,NaN,NaN,NaN,0.0,0.0,Staten Island,11/10/2011 02:50:52 PM,Failed for Other R,11/14/2011 03:28:40 PM,"(0.0, 0.0)"
2,Compliance,12990129,PC7054124,2,3089140016,3,8914,16,000,Abbey Court,NaN,0.0,0.0,0.0,0.0,Brooklyn,09/26/2017 02:00:25 PM,Passed,09/27/2017 12:58:55 PM,NaN
3,Initial,12328443,PC6737067,1,2025390132,2,2539,132,10,MAJ WM DEEGAN BLVD,NaN,1003704.0,240136.0,0.0,0.0,Bronx,09/12/2017 02:45:42 PM,Passed,09/14/2017 02:41:03 PM,"(0.0, 0.0)"
4,Initial,11362643,PC7466087,1,5002210035,5,221,35,000,Roe Street,NaN,950510.0,169320.0,0.0,0.0,Staten Island,06/02/2016 03:08:18 PM,Passed,06/07/2016 02:35:38 PM,"(0.0, 0.0)"


In [6]:
len(data)

2007833

**Al día 16 de noviembre de 2021 contamos con 2,007,833 datos**

In [7]:
data.columns

Index([u'INSPECTION_TYPE', u'JOB_TICKET_OR_WORK_ORDER_ID', u'JOB_ID',
       u'JOB_PROGRESS', u'BBL', u'BORO_CODE', u'BLOCK', u'LOT',
       u'HOUSE_NUMBER', u'STREET_NAME', u'ZIP_CODE', u'X_COORD', u'Y_COORD',
       u'LATITUDE', u'LONGITUDE', u'BOROUGH', u'INSPECTION_DATE', u'RESULT',
       u'APPROVED_DATE', u'LOCATION'],
      dtype='object')

Los tipos de dato que tiene cada variable inicialmente son:

In [8]:
data.dtypes

INSPECTION_TYPE                 object
JOB_TICKET_OR_WORK_ORDER_ID      int64
JOB_ID                          object
JOB_PROGRESS                     int64
BBL                              int64
BORO_CODE                        int64
BLOCK                            int64
LOT                              int64
HOUSE_NUMBER                    object
STREET_NAME                     object
ZIP_CODE                       float64
X_COORD                        float64
Y_COORD                        float64
LATITUDE                       float64
LONGITUDE                      float64
BOROUGH                         object
INSPECTION_DATE                 object
RESULT                          object
APPROVED_DATE                   object
LOCATION                        object
dtype: object

Cambiamos el formato de las columnas

def clean_column(col):
    """ Replace column's spaces and slash with underscore"""
    return col.lower()\
              .replace('/','_')\
              .replace(' ','_')

data.rename(columns={col: clean_column(col) for col in data.columns.values}, inplace=True)

## Perfilamiento general con los datos en crudo

In [9]:
eda.genera_profiling_general(data)

El dataframe tiene 20 columnas.
Hay 12 columnas que tienen NA's.


,Estadísticas,Resultado
0,Total de variables,20
1,Conteo de observaciones,2007833
2,Total de celdas,40156660
3,Cantidad de variables numéricas,11
4,Cantidad de variables de fecha,0
5,Cantidad de variables categóricas,0
6,Cantidad de variables de texto,9
7,Valores faltantes,334484
8,Porcentaje de valores faltantes,0%
9,Renglones duplicados,0


En la base de datos tenemos 334,484 elementos faltantes, distribuidos de la siguiente forma:

In [10]:
eda.cuenta_nulos_por_columnas(data)

El dataframe tiene 20 columnas.
Hay 12 columnas que tienen NA's.


,Missing Values,% del Total
X_COORD,123209,6.1
Y_COORD,123209,6.1
HOUSE_NUMBER,48758,2.4
ZIP_CODE,11154,0.6
LOCATION,7397,0.4
LATITUDE,6754,0.3
LONGITUDE,6754,0.3
STREET_NAME,5483,0.3
APPROVED_DATE,1633,0.1
BOROUGH,103,0.0


Cambiamos tipos de datos

In [11]:
data = data.astype({"INSPECTION_TYPE":'category',
                    "STREET_NAME":'category', 
                    "ZIP_CODE":'category', 
                    "BOROUGH":'category', 
                    "RESULT":'category'})
data['INSPECTION_DATE'] = pd.to_datetime(data['INSPECTION_DATE'], dayfirst=True)
data['APPROVED_DATE'] = pd.to_datetime(data['APPROVED_DATE'], dayfirst=True)

Generamos perfilamiento por tipo de variable

In [12]:
profiling_numericas, profiling_categoricas, profiling_texto, profiling_fechas = \
eda.genera_profiling_por_variable(data)

In [13]:
profiling_numericas

,Métrica,JOB_TICKET_OR_WORK_ORDER_ID,JOB_PROGRESS,BBL,BORO_CODE,BLOCK,LOT,X_COORD,Y_COORD,LATITUDE,LONGITUDE
0,Tipo,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64
1,Número de observaciones,2.00783e+06,2.00783e+06,2.00783e+06,2.00783e+06,2.00783e+06,2.00783e+06,1.88462e+06,1.88462e+06,2.00108e+06,2.00108e+06
2,Media,1.07449e+07,1.92,2.2368e+09,2.21,2579.37,193,1.01993e+06,233848,40.54,-73.54
3,Desviación estándar,3.52033e+06,3.71,1.08277e+09,1.07,1996.42,1029.68,1.93368e+06,1.89098e+06,2.98,5.4
4,Cuartil 25%,1.15108e+07,1,1.01922e+09,1,1396,17,995025,193617,40.7,-73.96
5,Cuartil 50%,1.20439e+07,1,2.03171e+09,2,2176,36,1.00301e+06,217984,40.76,-73.93
6,Cuartil 75%,1.25876e+07,2,3.0202e+09,3,3219,60,1.01229e+06,242724,40.83,-73.9
7,Mínimo,2.37227e+06,1,1e+09,1,0,0,0,0,-18.91,-79.47
8,Máximo,1.32944e+07,175,9.05361e+09,9,27000,9999,3.19139e+08,3.74392e+08,40.91,0
9,Número de observaciones únicas,2007833,175,296006,6,12368,1253,83782,111316,311211,311691


In [14]:
profiling_categoricas

,Métrica,INSPECTION_TYPE,STREET_NAME,ZIP_CODE,BOROUGH,RESULT
0,Tipo,category,category,category,category,category
1,Número de categorías,5,12082,229,5,7
2,Número de observaciones,2007833,2007833,2007833,2007833,2007833
3,Observaciones nulas,0,5483,11154,103,13
4,% Observaciones nulas,0,0,0,0,0
5,Valores únicos,"[Initial, Compliance, BAIT, STOPPAGE, CLEAN_UPS]","[MELROSE AVENUE, WENLOCK STREET, Abbey Court, ...","[nan, 10025.0, 11215.0, 10009.0, 10301.0, 1030...","[Bronx, Staten Island, Brooklyn, Manhattan, Qu...","[Failed for Other R, Passed, Rat Activity, Bai..."
6,Moda1/veces/%,"[Initial, 307,794, 15.33%]","[BROADWAY, 741, 0.04%]","[10457.0, 3,864, 0.19%]","[Manhattan, 596,696, 29.72%]","[Passed, 271,199, 13.51%]"
7,Moda2/veces/%,"[BAIT, 1,585, 0.08%]","[3 AVENUE, 4, 0.0%]","[11221.0, 1, 0.0%]","[Bronx, 531,773, 26.49%]","[Bait applied, 1,585, 0.08%]"
8,Moda3/veces/%,"[Compliance, 280,931, 13.99%]","[GRAND CONCOURSE, 1, 0.0%]","[10458.0, 1, 0.0%]","[Brooklyn, 641,019, 31.93%]","[Rat Activity, 197,036, 9.81%]"


In [15]:
profiling_texto

,Métrica,JOB_ID,HOUSE_NUMBER,LOCATION
0,Tipo,object,object,object
1,Número de observaciones,2007833,2007833,2007833
2,Observaciones únicas,1445313,21308,311840
3,% Observaciones únicas,0,0,0
4,Tamaño promedio,9,3.35165,34.6904
5,Tamaño mínimo,9,1,10
6,Tamaño máximo,9,47,38


In [16]:
profiling_fechas

,Métrica,INSPECTION_DATE,APPROVED_DATE
0,Tipo,datetime64[ns],datetime64[ns]
1,Número de observaciones,2007816,2006200
2,Mínimo,1918-10-19 14:34:44,2009-01-10 08:06:55
3,Máximo,2029-01-01 09:35:51,2021-12-11 17:14:04
4,Número de observaciones únicas,1951381,1077711
5,Número de faltantes,17,1633
6,Top1/veces/%,"[2011-01-10 12:00:00, 30, 0.0%]","[2010-08-13 16:09:45, 61, 0.0%]"
7,Top2/veces/%,"[2011-11-19 12:00:00, 28, 0.0%]","[2010-08-17 14:31:44, 52, 0.0%]"
8,Top3/veces/%,"[2011-12-11 12:00:00, 28, 0.0%]","[2010-08-27 15:08:45, 50, 0.0%]"


## Perfilamiento general haciendo limpieza de datos